In [21]:
import numpy as np
import pandas as pd
import os
import random
import copy

# Calculation of Relative Strength Index (RSI)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

dirname = os.path.abspath('')
data_path = os.path.join(dirname, 'Data')

stocks_df = pd.read_csv(data_path + '\\' + 'stocks_with_next_changes.csv', encoding='utf-8')

stocks = stocks_df['persian_symbol'].unique().tolist()
print(len(stocks))

373


In [25]:
indicators_df = pd.read_csv(data_path + '\\' + 'stocks_with_indicators.csv', encoding='utf-8')

print(indicators_df.columns)

done


In [22]:
stocks_df['Momentum_1D'] = 0

def rsi(values):
    up = values[values > 0].mean()
    down = -1 * values[values < 0].mean()
    return 100 * up / (up + down)

for stock in stocks:
    stocks_df.loc[stocks_df['persian_symbol'] == stock, 'Momentum_1D'] = (
        stocks_df.loc[stocks_df['persian_symbol'] == stock, 'final_price'] -
        stocks_df.loc[stocks_df['persian_symbol'] == stock, 'final_price'].shift(1)
    ).fillna(0)

    stocks_df.loc[stocks_df['persian_symbol'] == stock, 'RSI_5D'] = \
        stocks_df.loc[stocks_df['persian_symbol'] == stock, 'Momentum_1D'].rolling(center=False, window=5). \
        apply(rsi).fillna(0)

stocks_df.to_csv(
    data_path + '\\' + 'stocks_with_indicators.csv', encoding='utf-8-sig', index=False)

print("done")

done


In [4]:
# Calculation of Volume (Plain)

indicators_df['Volume_plain'] = indicators_df['volume'].fillna(0)

indicators_df.to_csv(
    data_path + '\\' + 'stocks_with_indicators.csv', encoding='utf-8-sig', index=False)


In [6]:
# Calculation of Bollinger Bands
def bbands(price, length=30, numsd=2):
    """ returns average, upper band, and lower band"""
    #ave = pd.stats.moments.rolling_mean(price,length)
    ave = price.rolling(window = length, center = False).mean()
    #sd = pd.stats.moments.rolling_std(price,length)
    sd = price.rolling(window = length, center = False).std()
    upband = ave + (sd*numsd)
    dnband = ave - (sd*numsd)
    return np.round(ave,3), np.round(upband,3), np.round(dnband,3)

indicators_df['BB_Middle_Band'], indicators_df['BB_Upper_Band'], indicators_df['BB_Lower_Band'] = \
    bbands(indicators_df['final_price'], length=20, numsd=1)

indicators_df['BB_Middle_Band'] = indicators_df['BB_Middle_Band'].fillna(0)
indicators_df['BB_Upper_Band'] = indicators_df['BB_Upper_Band'].fillna(0)
indicators_df['BB_Lower_Band'] = indicators_df['BB_Lower_Band'].fillna(0)
   
indicators_df.to_csv(
    data_path + '\\' + 'stocks_with_indicators.csv', encoding='utf-8-sig', index=False)


In [9]:
# Calculation of Aroon Oscillator
def aroon(df, tf=25):
    aroonup = []
    aroondown = []
    x = tf
    while x< len(df['pd_trade_date']):
        aroon_up = ((df['max_price'][x-tf:x].tolist().index(max(df['max_price'][x-tf:x])))/float(tf))*100
        aroon_down = ((df['min_price'][x-tf:x].tolist().index(min(df['min_price'][x-tf:x])))/float(tf))*100
        aroonup.append(aroon_up)
        aroondown.append(aroon_down)
        x+=1
    return aroonup, aroondown


listofzeros = [0] * 25
up, down = aroon(indicators_df)
aroon_list = [x - y for x, y in zip(up,down)]
if len(aroon_list)==0:
    aroon_list = [0] * indicators_df.shape[0]
    indicators_df['Aroon_Oscillator'] = aroon_list
else:
    indicators_df['Aroon_Oscillator'] = listofzeros+aroon_list

indicators_df.to_csv(
    data_path + '\\' + 'stocks_with_indicators.csv', encoding='utf-8-sig', index=False)

In [10]:
# Calculation of Price Volume Trend
# PVT = [((CurrentClose - PreviousClose) / PreviousClose) x Volume] + PreviousPVT
indicators_df["PVT"] = (indicators_df['Momentum_1D']/ indicators_df['final_price'].shift(1))*indicators_df['volume']
indicators_df["PVT"] = indicators_df["PVT"]-indicators_df["PVT"].shift(1)
indicators_df["PVT"] = indicators_df["PVT"].fillna(0)

indicators_df.to_csv(
    data_path + '\\' + 'stocks_with_indicators.csv', encoding='utf-8-sig', index=False)

In [13]:
# Calculation of Acceleration Bands
def abands(df):
    #df['AB_Middle_Band'] = pd.rolling_mean(df['final_price'], 20)
    df['AB_Middle_Band'] = df['final_price'].rolling(window = 20, center=False).mean()
    # High * ( 1 + 4 * (High - Low) / (High + Low))
    df['aupband'] = df['max_price'] * (1 + 4 * (df['max_price']-df['min_price'])/(df['max_price']+df['min_price']))
    df['AB_Upper_Band'] = df['aupband'].rolling(window=20, center=False).mean()
    # Low *(1 - 4 * (High - Low)/ (High + Low))
    df['adownband'] = df['min_price'] * (1 - 4 * (df['max_price']-df['min_price'])/(df['max_price']+df['min_price']))
    df['AB_Lower_Band'] = df['adownband'].rolling(window=20, center=False).mean()

abands(indicators_df)
indicators_df = indicators_df.fillna(0)

indicators_df.to_csv(
    data_path + '\\' + 'stocks_with_indicators.csv', encoding='utf-8-sig', index=False)

In [14]:
# Drop unwanted columns
columns2Drop = ['aupband', 'adownband']
indicators_df = indicators_df.drop(labels = columns2Drop, axis=1)

indicators_df.to_csv(
    data_path + '\\' + 'stocks_with_indicators.csv', encoding='utf-8-sig', index=False)


In [6]:
# Calculation of Stochastic Oscillator (%K and %D)
print(":0:1:0:1:0:0:1:1:0:1:0:0:0:0:0:0:0:0")
def STOK(df, n):
    df['STOK'] = ((df['final_price'] - df['min_price'].rolling(window=n, center=False).mean()) /
            (df['max_price'].rolling(window=n, center=False).max() - df['min_price'].rolling(window=n, center=False).min())) * 100
    df['STOD'] = df['STOK'].rolling(window = 3, center=False).mean()

STOK(indicators_df, 4)
indicators_df = indicators_df.fillna(0)

indicators_df.to_csv(
    data_path + '\\' + 'stocks_with_indicators.csv', encoding='utf-8-sig', index=False)

:0:1:0:1:0:0:1:1:0:1:0:0:0:0:0:0:0:0


In [4]:
print("Calculation of Chaikin Money Flow")
def CMFlow(df, tf):
    CHMF = []
    MFMs = []
    MFVs = []
    x = tf

    while x < len(df['pd_trade_date']):
        PeriodVolume = 0
        volRange = df['volume'][x-tf:x]
        for eachVol in volRange:
            PeriodVolume += eachVol

        MFM = ((df['final_price'][x] - df['min_price'][x]) -
               (df['max_price'][x] - df['final_price'][x])) / (df['max_price'][x] - df['min_price'][x])
        MFV = MFM*PeriodVolume

        MFMs.append(MFM)
        MFVs.append(MFV)
        x+=1

    y = tf
    while y < len(MFVs):
        PeriodVolume = 0
        volRange = df['volume'][x-tf:x]
        for eachVol in volRange:
            PeriodVolume += eachVol
        consider = MFVs[y-tf:y]
        tfsMFV = 0

        for eachMFV in consider:
            tfsMFV += eachMFV

        tfsCMF = tfsMFV/PeriodVolume
        CHMF.append(tfsCMF)
        y+=1
    return CHMF

listofzeros = [0] * 40
CHMF = CMFlow(indicators_df, 20)
if len(CHMF)==0:
    CHMF = [0] * indicators_df.shape[0]
    indicators_df['Chaikin_MF'] = CHMF
else:
    indicators_df['Chaikin_MF'] = listofzeros+CHMF

indicators_df.to_csv(
    data_path + '\\' + 'stocks_with_indicators2.csv', encoding='utf-8-sig', index=False)

print("passed")


In [10]:
print("Calculation of Parabolic SAR .")


def psar(df, iaf = 0.02, maxaf = 0.2):
    length = len(df)
    dates = (df['pd_trade_date'])
    high = (df['max_price'])
    low = (df['min_price'])
    close = (df['final_price'])
    psar = df['final_price'][0:len(df['final_price'])]
    psarbull = [None] * length
    psarbear = [None] * length
    bull = True
    af = iaf
    ep = df['min_price'][0]
    hp = df['max_price'][0]
    lp = df['min_price'][0]
    for i in range(2,length):
        if bull:
            psar[i] = psar[i - 1] + af * (hp - psar[i - 1])
        else:
            psar[i] = psar[i - 1] + af * (lp - psar[i - 1])
        reverse = False
        if bull:
            if df['min_price'][i] < psar[i]:
                bull = False
                reverse = True
                psar[i] = hp
                lp = df['min_price'][i]
                af = iaf
        else:
            if df['max_price'][i] > psar[i]:
                bull = True
                reverse = True
                psar[i] = lp
                hp = df['max_price'][i]
                af = iaf
        if not reverse:
            if bull:
                if df['max_price'][i] > hp:
                    hp = df['max_price'][i]
                    af = min(af + iaf, maxaf)
                if df['min_price'][i - 1] < psar[i]:
                    psar[i] = df['min_price'][i - 1]
                if df['min_price'][i - 2] < psar[i]:
                    psar[i] = df['min_price'][i - 2]
            else:
                if df['min_price'][i] < lp:
                    lp = df['min_price'][i]
                    af = min(af + iaf, maxaf)
                if df['max_price'][i - 1] > psar[i]:
                    psar[i] = df['max_price'][i - 1]
                if df['max_price'][i - 2] > psar[i]:
                    psar[i] = df['max_price'][i - 2]
        if bull:
            psarbull[i] = psar[i]
        else:
            psarbear[i] = psar[i]

    df['psar'] = psar

psar(indicators_df)

indicators_df.to_csv(
    data_path + '\\' + 'stocks_with_indicators.csv', encoding='utf-8-sig', index=False)

Calculation of Parabolic SAR .


In [11]:
# Drop Unwanted columns
columns2Drop = ['Chaikin_MF']
indicators_df = indicators_df.drop(labels = columns2Drop, axis=1)

indicators_df.to_csv(
    data_path + '\\' + 'stocks_with_indicators.csv', encoding='utf-8-sig', index=False)


In [13]:
print('Calculation of Price Rate of Change')
# ROC = [(Close - Close n periods ago) / (Close n periods ago)] * 100
indicators_df['ROC'] = ((indicators_df['final_price'] - indicators_df['final_price'].shift(12))/(indicators_df['final_price'].shift(12)))*100
indicators_df = indicators_df.fillna(0)

indicators_df.to_csv(
    data_path + '\\' + 'stocks_with_indicators.csv', encoding='utf-8-sig', index=False)

Calculation of Price Rate of Change


In [14]:
print('Calculation of Volume Weighted Average Price')

indicators_df['VWAP'] = np.cumsum(indicators_df['volume'] * (indicators_df['max_price'] + indicators_df['min_price'])/2) / np.cumsum(indicators_df['volume'])
indicators_df = indicators_df.fillna(0)

indicators_df.to_csv(
    data_path + '\\' + 'stocks_with_indicators.csv', encoding='utf-8-sig', index=False)


Calculation of Volume Weighted Average Price
